In [1]:
import restoration as res

c:\Users\Salvos\anaconda3\envs\multi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Salvos\anaconda3\envs\multi\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QFileDialog
import cv2
import os
from PyQt6.QtCore import QTimer  # Importa QTimer per la gestione del tempo

class MyApp(QWidget):
    def __init__(self):
        super().__init__()
        
        self.adjustSize()
        self.layout = QVBoxLayout()
        self.label = QLabel("")
        self.layout.addWidget(self.label)

        # Pulsante per importare il video
        self.button1 = QPushButton("Importa Video")
        self.button1.clicked.connect(self.import_video)
        self.layout.addWidget(self.button1)

        # Pulsante per dividere in frame (inizialmente nascosto)
        self.button2 = QPushButton("Dividi in Frame")
        self.button2.clicked.connect(self.split_frames)
        self.button2.setVisible(False)
        self.layout.addWidget(self.button2)
        
        # Pulsante per ulteriori operazioni dopo la divisione (inizialmente nascosto)
        self.button3 = QPushButton("High Res")
        self.button3.clicked.connect(self.high_res)
        self.button3.setVisible(False)
        self.layout.addWidget(self.button3)

        self.button4 = QPushButton("Genera video")
        self.button4.clicked.connect(self.create_video)
        self.button4.setVisible(True) #da cambiare 
        self.layout.addWidget(self.button4)

        self.video_path = None
        
        self.setLayout(self.layout)
        self.setWindowTitle("Gestione Video")

    def import_video(self):
        self.button3.setVisible(False)
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Seleziona un video", "", "Video Files (*.mp4 *.avi *.mov)")
        
        if file_path:
            self.video_path = file_path
            self.label.setText(f"Video importato: {os.path.basename(file_path)}")
            self.button2.setVisible(True)  # Mostra il pulsante per dividere in frame
    
    def split_frames(self):
        if not self.video_path:
            self.label.setText("Nessun video selezionato!")
            return
        
        cap = cv2.VideoCapture(self.video_path)
        frame_count = 0
        output_folder = f"frames/{os.path.basename(self.video_path)}"
        os.makedirs(output_folder, exist_ok=True)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
        
        cap.release()
        self.label.setText(f"Video diviso in {frame_count} frame salvati in '{output_folder}'")
        self.button2.setVisible(False)
        self.button3.setVisible(True)  
    
    def high_res(self):
        self.label.setText(f"Funzione High Res in esecuzione...")
        QTimer.singleShot(1000, self.apply_high_res)  

    def apply_high_res(self):
        video_name = os.path.basename(self.video_path)
        res.hight_res(self.label, folder_frames=f"frames/{video_name}", save_dir= f'results/swinir_{video_name})
        self.label.setText("Funzione High Res completata!")

    def create_video(self):
        self.label.setText("Creazione video ...")
        folder=f"frames/{os.path.basename(self.video_path)}"
        res.reconstruct_video_from_frames(frame_dir=folder)
        self.label.setText("Creazione video completata!")
        self.button4.setVisible(False)  



if __name__ == "__main__":
    app = QApplication([])
    window = MyApp()
    window.show()
    app.exec()
